In [1]:
import re
import os
import shutil
from pathlib import Path

In [2]:
def search_report(pattern, filename):
    match = re.search(pattern, filename)
    if match:
        report_id = match.group(1)
        return report_id
    else:
        return filename

def txt_file(filepath):
    if filepath.endswith('TOTAL.txt'):
        pass
    else:
        return filepath.lower().endswith('.txt')
    
def imgfile(filepath):
    if filepath.lower().endswith('.jpg'):
        return True


def makedir(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    # change permission into read, write, execute
    Path(path).chmod(0o777)

In [3]:
base_dir = os.path.join('..','datasets','radiology_clean', 'Thorax', '231030') # ganti folder disini
separated_report_dir = os.path.join('..', 'separated_report', 'Thorax', 'Thorax-231030-report-1') # ganti nama file gabungan report disini
target_dir = os.path.join('..', 'datasets', 'test_thorax_clean')

image_list = []

# append all images in the folder
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if imgfile(file):
            image_list.append(file)
        


In [4]:

image_file_list = []
missing_report = []

for root, dirs, files in os.walk(separated_report_dir):
    for file in files:
        # get report id
        pattern_separated_report = r'Thorax-(\d+)-*'
        report_id = search_report(pattern_separated_report, file)  

        # get image id
        pattern_image = r'Thorax-231030-2310300(\d+)-*' # ganti pattern nama filenya sblm idnya disini
        for image_file in image_list:
            image_id = search_report(pattern_image, image_file)
            if image_id.startswith('0'):
                image_id = image_id[1:]

            source_dir = ''
            dest_dir = ''
            
            # if match with report id
            if image_id == report_id:
                pattern = r'^.*(?=-[^-]*$)'
                target_folder = re.search(pattern, image_file).group()

                # duplicate the image to the new folder
                source_dir = os.path.join(base_dir, image_file)
                dest_dir = os.path.join(target_dir, target_folder)
                makedir(dest_dir)
                shutil.copy2(source_dir, dest_dir)

                # duplicate the txt file
                source_dir = os.path.join(separated_report_dir, file)
                shutil.copy2(source_dir, dest_dir)

                # rename txt file into correct format
                new_txt_filepath = os.path.join(dest_dir, file)
                os.rename(new_txt_filepath, os.path.join(dest_dir, target_folder+'-report-1.txt'))

                image_file_list.append(image_file)


for i in image_file_list:
    print(i)


Thorax-231030-2310300113_Series_1001_0000-23.jpg
Thorax-231030-2310300117_Series_1001_0000-24.jpg
Thorax-231030-2310300118_Series_1001_0000-25.jpg
Thorax-231030-2310300119_Series_1001_0000-26.jpg
Thorax-231030-2310300121_Series_1001_0000-27.jpg
Thorax-231030-2310300126_Series_1001_0000-28.jpg
Thorax-231030-2310300127_Series_1001_0000-29.jpg
Thorax-231030-2310300012_Series_1001_0000-1.jpg
Thorax-231030-2310300130_Series_1001_0000-30.jpg
Thorax-231030-2310300134_Series_1001_0000-31.jpg
Thorax-231030-2310300135_Series_1001_0000-32.jpg
Thorax-231030-2310300138_Series_1001_0000-33.jpg
Thorax-231030-2310300139_Series_1001_0000-34.jpg
Thorax-231030-2310300140_Series_1001_0000-35.jpg
Thorax-231030-2310300141_Series_1001_0000-36.jpg
Thorax-231030-2310300142_Series_1001_0000-37.jpg
Thorax-231030-2310300143_Series_1001_0000-38.jpg
Thorax-231030-2310300145_Series_1001_0000-39.jpg
Thorax-231030-2310300146_Series_1001_0000-40.jpg
Thorax-231030-2310300147_Series_1001_0000-41.jpg
Thorax-231030-2310300

In [7]:
missing_report = []

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file not in image_file_list and txt_file(file) != True:
            missing_report.append(file)
print('missing report', missing_report)

missing report []
